# A Policy Scenario

In this notebook, we will step through creating and investigating a policy scenario.

## The Scenario

We will look into the effect of wind generation subsidies on the electricity sector. These subsidies will take the form of capital investment subsidies, effectively reducing the costs of capital investment.

# Setup and Helper Variables

In [ ]:
# load required packages 
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

from ixmp import ixPlatform
from tools import Plots

In [ ]:
# launch the IX modeling platform using the local default database                                                                                                                       
mp = ixPlatform(dbtype='HSQLDB') 

In [ ]:
country = 'Austria'

plants = [
    "coal_ppl", 
    "gas_ppl", 
    "oil_ppl", 
    "bio_ppl", 
    "hydro_ppl",
    "wind_ppl", 
    "solar_pv_ppl", # actually primary -> final
]

lights = [
    "bulb", 
    "cfl", 
]

# Create the New Policy Scenario Datastructure

This operation is done by cloning the baseline scenario, which will then be edited.

In [ ]:
model = "Austrian energy model"
scen = "baseline"
mp.open_db() # only open if you're on a local database!
base_ds = mp.datastructure(model, scen)
ds = base_ds.clone(model, 'wind subsidies', 'testing wind subsidies', keep_sol=False)
ds.check_out()

In [ ]:
horizon = ds.set('year')

# Edit the Datastructure with the Scenario Parameters

Introduce subsidies that decay with time. Initially they are 50% of the total investment costs and decay to 0.

In [ ]:
ds.par('inv_cost').tail()

In [ ]:
subsidies = np.array([0.5, 0.5, 0.75, 1])
inv_cost = pd.DataFrame({
        'node_loc': country,
        'year_vtg': horizon,
        'technology': 'wind_ppl',
        'value': 1100 * subsidies * 1e6,
        'unit': 'USD/GWa',
})
ds.add_par('inv_cost', inv_cost)

In [ ]:
ds.par('inv_cost').tail()

# Solve the Model

In [ ]:
ds.commit('wind subsidies added')
ds.set_as_default()
ds.solve()

# Investigate the Results

In [ ]:
p = Plots(ds, country)
p.plot_new_capacity(baseyear=True, subset=plants)
p.plot_capacity(baseyear=True, subset=plants)
p.plot_activity(baseyear=True, subset=plants)

In [ ]:
mp.close_db()